### Data Loading

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df=pd.DataFrame(
    columns = ['build_Failed', 'gh_is_pr', 'git_prev_commit_resolution_status',
       'gh_team_size', 'gh_num_commit_comments', 'git_diff_src_churn',
       'git_diff_test_churn', 'gh_diff_files_added', 'gh_diff_files_deleted',
       'gh_diff_files_modified', 'gh_diff_tests_added',
       'gh_diff_tests_deleted', 'gh_diff_src_files', 'gh_diff_doc_files',
       'gh_diff_other_files', 'gh_sloc', 'gh_test_lines_per_kloc',
       'gh_test_cases_per_kloc', 'gh_asserts_cases_per_kloc', 'tr_build_id',
       'gh_build_started_at'],
    dtype='object')

path = 'D:\PFE\Code\CI-Failing-Builds-Detection\dataset'
path = '/mnt/d/PFE/Code/CI-Failing-Builds-Detection/dataset'

for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename[-4:]==".csv":
            df = pd.concat([df, pd.read_csv(os.path.join(dirname, filename))])

X = df.iloc[:,1:19]
y = df.iloc[:,0].astype(int)

from sklearn.model_selection import train_test_split
X_train, X_val , y_train, y_val = train_test_split(np.array(X), np.array(y), test_size=0.2, shuffle=True, stratify=y, random_state=42) # keep ratio of classes in split

### Modifiable Decision Tree model 

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import _tree
from sklearn.metrics import f1_score 

class modDecisionTree:
    """
    Represents the classification model
    based on sklearn implementation with added methods for modifying single nodes
    nodes are indexed depth first
    """

    def __init__(self, max_depth=3, random_state=42):
        # need to add init of hyper-param
        self.model = DecisionTreeClassifier(max_depth=max_depth, splitter='random', random_state=random_state)
    
    def fit(self, X_train, y_train, columns_names):
        self.model.fit(X_train, y_train)
        self.tree = self.model.tree_
        self.n_nodes = self.tree.node_count       # nbr of nodes
        self.features = self.tree.feature         # list of each node's feature
        self.thresholds = self.tree.threshold     # list of each node's threshold
        self.columns_names = columns_names
        self.features_names = [list(self.columns_names)[i] for i in self.features]
        self.nodes_type = self.get_nodes_type()  # a node is either a split node or a terminal node/leaf

    def evaluate(self, X_val, y_val):
        y_pred = self.model.predict(X_val)
        metrics = {}
        metrics['F1'] = f1_score(y_pred ,y_val)
        return metrics

    def feature_importance(self):
        feat_imp = []
        for name, importance in zip(self.features_names , self.model.feature_importances_):
            feat_imp.append((name, importance))
        feat_imp.sort(key=lambda t:t[1], reverse=True)
        return feat_imp

    def plot_tree(self):
        plt.figure(figsize=(15,10))  # set plot size (denoted in inches)
        tree.plot_tree(self.model, fontsize=10, class_names=['pass','fail'])
        plt.show()
    
    def get_nodes_type(self):
        children_left = self.tree.children_left
        children_right = self.tree.children_right
        node_depth = np.zeros(shape=self.n_nodes, dtype=np.int64)
        is_leaves = np.zeros(shape=self.n_nodes, dtype=bool)
        stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
        while len(stack) > 0:
            # 'pop' ensures each node is only visited once
            node_id, depth = stack.pop()
            node_depth[node_id] = depth
            # If the left and right child of a node is not the same we have a split node
            is_split_node = children_left[node_id] != children_right[node_id]
            # If a split node, append left and right children and depth to `stack`
            # so we can loop through them
            if is_split_node:
                stack.append((children_left[node_id], depth + 1))
                stack.append((children_right[node_id], depth + 1))
            else:
                is_leaves[node_id] = True
        return is_leaves

    def node_is_leaf(self, node):
        """Returns whether a node is a leaf (terminal node) or a split node"""
        return self.nodes_type[node]==1

    def set_node_threshold(self, node, value):
        if node>=self.tree.node_count:
            print("Error: selected node id is not in the tree.")
            return
        if self.node_is_leaf(node):
            print("Error: can't change a leaf node's threshold.")
            return
        self.threshold[node] = value 

    def set_node_feature(self, node, feat_index=None, feat_name=None):
        if node>=self.tree.node_count:
            print("Error: selected node id is not in the tree.")
            return
        if self.node_is_leaf(node):
            print("Error: can't change a terminal node's feature.")
            return
        # convert feature name to index if supplied with name
        if feat_index==None:
            feat_index = self.features_names.index(feat_name)
        self.features[node] = feat_index

### State Generation

In [3]:
def tree_convolution(model, features, thresholds, node=0):
    """
        convolution of each 3 nodes, with overlapping => child of 1 subtree is parent of the next
        => after each conv, slit nodes' featuress and thresholdss are updated and terminal nodes are removed
        returns new features,thresholds values as lists
    """
    visited = []
    queue = []
    visited.append(node)
    queue.append(node)
    new_features = features
    new_thresholds = thresholds
    #replacing leaf default values -2 by 0 for conv
    for node in range(model.n_nodes): 
        if new_features[node]==-2:
            new_features[node]=0
        if new_thresholds[node]==-2:
            new_thresholds[node]=0
    # BFS traversal
    while queue:
        node = queue.pop(0) 
        left_node = model.tree.children_left[node]
        right_node = model.tree.children_right[node]
        #print (node, left_node, right_node, end = "\n")
        if left_node not in visited:
            visited.append(left_node)
            queue.append(left_node)
        if right_node not in visited:
            visited.append(right_node)
            queue.append(right_node)

        # define kernel here
        if features[left_node]==0 and features[right_node]==0: # both child nodes are leaves
            new_node_features = 0
            new_node_thresholds = 0
        elif features[left_node]==0: # left node is leaf
            new_node_features = (features[node]+features[right_node])/2
            new_node_thresholds = (thresholds[node]+thresholds[right_node])/2
        elif features[right_node]==0: # right node is leaf
            new_node_features = (features[node]+features[left_node])/2
            new_node_thresholds = (thresholds[node]+thresholds[left_node])/2
        else:
            new_node_features = (features[node]+features[left_node]+features[right_node])/3
            new_node_thresholds = (thresholds[node]+thresholds[left_node]+thresholds[right_node])/3
        # save new values
        new_features[node] = new_node_features # will auto convert to int 
        new_thresholds[node] = new_node_thresholds

    return new_features, new_thresholds

import torch
def flatten(features, thresholds):
    # removing null values
    clean_feat, clean_thres = [], []
    for i in range(len(features)):
        if features[i]!=0:
            clean_feat.append(features[i])
            clean_thres.append(thresholds[i])
    return torch.cat((torch.FloatTensor(clean_feat), torch.FloatTensor(clean_thres)))


def generate_state(model, features, thresholds, nbr_of_conv):
    features, thresholds = features.copy(), thresholds.copy()
    for _ in range(nbr_of_conv):
        new_features, new_thresholds = tree_convolution(model, features, thresholds) # model must be fitted => add condition?
        features, thresholds = new_features, new_thresholds
    return flatten(features, thresholds)


In [4]:
import numpy as np
import random

from collections import namedtuple, deque
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import time
from torch.utils.tensorboard import SummaryWriter
import argparse


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

In [8]:
class ThresholdsNetwork(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, threshold_vector_size, seed, hidden_size=32):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            threshold_vector_size (int): Dimension of each action
            seed (int): Random seed
            hidden_size (int): Number of nodes in hidden layers
        """
        super(ThresholdsNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)     
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, threshold_vector_size)

    def forward(self, state):
        x = F.relu(self.fc1(state), inplace=True)
        x = F.relu(self.fc2(x), inplace=True)
        out = self.fc3(x)
        return out
    
    def get_thresholds_vector(self, state):
        """
        returns the action based on a squashed gaussian policy. That means the samples are obtained according to:
        a(s,e)= tanh(mu(s)+sigma(s)+e)
        """
        #state = torch.FloatTensor(state).to(device) #.unsqzeeze(0)
        #action = torch.clamp(action*action_high, action_low, action_high)
        
        threshold_vector = self.forward(state)
        return threshold_vector

### Reinforcement Learning Approach

In [9]:
class AttributeNetwork(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, threshold_vector_size, number_of_attributes, seed, hidden_size=32):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            threshold_vector_size (int): Dimension of each threshold vector
            seed (int): Random seed
            hidden_size (int): Number of nodes in the network layers

        """
        super(AttributeNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size+threshold_vector_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, number_of_attributes)

    def forward(self, state, threshold_vector):
        """Build a critic (value) network that maps (state, threshold_vector) pairs -> Q-values."""
        x = torch.cat((state, threshold_vector), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

    def get_attribute(self, state, threshold_vector):
        index_selected_attribute = np.argmax(self.forward(state, threshold_vector)) # check argmax output
        return index_selected_attribute

In [ ]:
class Agent():
    """Interacts with and learns from the environment."""
    
    def __init__(self, state_size, threshold_vector_size, number_of_attributes, random_seed, hidden_size):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            threshold_vector_size (int): dimension of each threshold vector
            random_seed (int): random seed
            add rest
        """
        self.state_size = state_size
        self.threshold_vector_size = threshold_vector_size
        self.number_of_attributes = number_of_attributes
        self.seed = random.seed(random_seed)
        print("Using: ", device)

        # actor Network 
        self.ThresholdsNetwork = ThresholdsNetwork(state_size, threshold_vector_size, random_seed, hidden_size).to(device)
        self.optimizer_ThresholdsNetwork = optim.Adam(self.ThresholdsNetwork.parameters(), lr=LR_ACTOR)     
        
        # critic Network  
        self.AttributeNetwork = AttributeNetwork(state_size, threshold_vector_size, number_of_attributes, random_seed, hidden_size).to(device)
        self.optimizer_AttributeNetwork = optim.Adam(self.AttributeNetwork.parameters(), lr=LR_CRITIC, weight_decay=0)

        # Replay memory
        self.memory = ReplayBuffer(BUFFER_SIZE, BATCH_SIZE, random_seed)
        

    def step(self, state, action, reward, next_state, done, step):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        self.memory.add(state, action, reward, next_state, done)

        # Learn, if enough samples are available in memory
        if len(self.memory) > BATCH_SIZE:
            experiences = self.memory.sample()
            self.learn(step, experiences, GAMMA)
            
    
    def act(self, state, node, eps=0.2):
        """Returns actions for given state as per current policy."""
        # greedy epsilon with param eps
        state = torch.from_numpy(state).float().to(device)
        thresholds_vector = self.ThresholdsNetwork.get_thresholds_vector(state)

        p = np.random.random() 
        if p<eps:
            index_selected_attribute = random.choice(self.number_of_attributes)
        else:
            index_selected_attribute = self.AttributeNetwork.get_attribute(state, thresholds_vector)
        action = (index_selected_attribute, thresholds_vector[index_selected_attribute])

        return action

    def learn(self, step, experiences, gamma):
        """Updates actor, critics using given batch of experience tuples.
        Critic_loss = 
        Actor_loss = 
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value
        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences
        # yb calc
        

        # ---------------------------- update thresholds network ---------------------------- #
        # Compute loss
        loss_thresholds_network = 
        # Minimize the loss
        self.optimizer_ThresholdsNetwork.zero_grad()
        loss_thresholds_network.backward()
        self.optimizer_ThresholdsNetwork.step()

        # ---------------------------- update attribute network ---------------------------- #
        # Compute loss
        loss_attribute_network = 

        # Minimize the loss
        self.optimizer_AttributeNetwork.zero_grad()
        loss_attribute_network.backward()
        self.optimizer_AttributeNetwork.step()


def env_step(model, node, action):
    # update tree
    model.set_node_feature(node, feat_index=action[0])
    model.set_node_threshold(node, value=action[1])

    next_state = generate_state(model, model.features, model.thresholds, nbr_of_conv)

    metrics = model.evaluate(X_val, y_val)
    reward = metrics['F1']

    done = 0
    info = 0

    return next_state, reward, done, info 

In [ ]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
        
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)
    

In [ ]:
seed = 42
GAMMA = 0.99
TAU = 1e-2
HIDDEN_SIZE = 256
BUFFER_SIZE = int(1e6)
BATCH_SIZE = 256
LR_ACTOR = 5e-4
LR_CRITIC = 5e-4
nbr_of_conv = 2

n_episodes=200
max_t=500
print_every=10

writer = SummaryWriter("runs/")
torch.manual_seed(seed)
np.random.seed(seed)
state_size = 
threshold_vector_size = 
number_of_attributes = 


t0 = time.time()

agent = Agent(state_size=state_size, threshold_vector_size=threshold_vector_size, number_of_attributes=number_of_attributes, random_seed=seed,hidden_size=HIDDEN_SIZE, action_prior="uniform") #"normal"

scores_deque = deque(maxlen=100)

for i_episode in range(1, n_episodes+1):
    # state reset
    model = modDecisionTree()
    model.fit(X_train, y_train, df.columns) 
    state = generate_state(model, model.features, model.thresholds, nbr_of_conv)
    #state = state.reshape((1,state_size))
    avg_score = 0
    
    for t in range(max_t):

        action = agent.act(state, node=t)
        
        next_state, reward, done, info = env_step(model, t, action)
        next_state = next_state.reshape((1,state_size))

        agent.step(state, action, reward, next_state, done, t)
        
        state = next_state

        avg_score += reward

        if done:
            break

    avg_score /= max_t

    scores_deque.append(avg_score)
    writer.add_scalar("Reward", avg_score, i_episode)
    
    print('\rEpisode {} Score: {:.2f}'.format(i_episode, avg_score, end="")
    if i_episode % print_every == 0:
        print('\rEpisode {} Score: {:.2f}'.format(i_episode, avg_score, end="")            
        
torch.save(agent.actor_local.state_dict(), args.info + ".pt")

t1 = time.time()
print("training took {} min!".format((t1-t0)/60))
